In [1]:
# Author: Vanshika Gupta
# Jointly with: Prof Chrysafis Vogiatzis
# Acknowledgement: https://neos-guide.org/content/sudoku

In [2]:
from gurobipy import *
import matplotlib.pyplot as plt
import numpy as np

#### Data

In [3]:
# dimension of outer Sudoku
nD = 9

# dimension of smaller box
nd = 3

# Predefined Matrix
preS = [(0, 0 ,6, 0, 8, 0, 5, 0, 0),
        (9, 0, 0, 3, 0, 0, 0, 2, 0),
        (0, 4, 0, 0, 0, 1, 0, 0, 7), 
        (0, 0, 0, 0, 0, 5, 6, 0, 0),
        (0, 2, 0, 0, 4, 0, 0, 1, 0),
        (3, 0, 0, 0, 0, 0, 0, 0, 9),
        (0, 0, 1, 9, 0, 0, 0, 3, 0),
        (0, 0, 0, 0, 2, 0, 4, 0, 0),
        (0, 5, 0, 0, 0, 7, 0, 0, 0)]

#### Variables

In [4]:
# final configuration of Sudoku
postS = {}

#### Create Model & Variables

In [5]:
model=Model("Sudoku")

postS = model.addVars(nD, nD, nD, vtype = GRB.BINARY)

Academic license - for non-commercial use only - expires 2022-09-08
Using license file /home/vanshika/gurobi.lic


#### Constraint

In [6]:
# preDefined Matrix should be same
for i in range(nD):
    for j in range(nD):
        if(preS[i][j]!=0):
            model.addConstr(postS[i,j,preS[i][j]-1] == 1)

# projection of 3-D on X-Z plane is ones(9x9)
for i in range(nD):
    for j in range(nD):
        expr = 0
        for k in range(nD):
            expr += postS[k,i,j]
        model.addConstr(expr == 1)
        
# projection of 3-D on Y-Z plane is ones(9x9)
for i in range(nD):
    for j in range(nD):
        expr = 0
        for k in range(nD):
            expr += postS[i,k,j]
        model.addConstr(expr == 1)
        
# no cell is left empty
for i in range(nD):
    for j in range(nD):
        expr = 0
        for k in range(nD):
            expr += postS[i,j,k]
        model.addConstr(expr == 1)
        
# projection of 3x3 box when opened along one direction is vector of 1s
for n in range(nd):
    for m in range(nd):
        for k in range(nD):
            expr = 0
            for r in range(nd):
                for s in range(nd):
                    expr += postS[3*n+r, 3*m+s, k]
            model.addConstr(expr == 1)

#### Optimize

In [7]:
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 347 rows, 729 columns and 2939 nonzeros
Model fingerprint: 0xc1023a14
Variable types: 0 continuous, 729 integer (729 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 347 rows and 729 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 16 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%


#### Print FInal Solution

In [8]:
print("Predefined Sudoku: \n")
print(np.asarray(preS), '\n')

p = np.ones( (9, 9),  dtype=np.int32 )
for i in range(nD):
    for j in range(nD):
        for k in range(nD):
            if(postS[i,j,k].X==1):
                p[i,j]=k+1

print("Solved Sudoku: \n")
print(p)

Predefined Sudoku: 

[[0 0 6 0 8 0 5 0 0]
 [9 0 0 3 0 0 0 2 0]
 [0 4 0 0 0 1 0 0 7]
 [0 0 0 0 0 5 6 0 0]
 [0 2 0 0 4 0 0 1 0]
 [3 0 0 0 0 0 0 0 9]
 [0 0 1 9 0 0 0 3 0]
 [0 0 0 0 2 0 4 0 0]
 [0 5 0 0 0 7 0 0 0]] 

Solved Sudoku: 

[[1 3 6 7 8 2 5 9 4]
 [9 8 7 3 5 4 1 2 6]
 [5 4 2 6 9 1 3 8 7]
 [7 9 8 1 3 5 6 4 2]
 [6 2 5 8 4 9 7 1 3]
 [3 1 4 2 7 6 8 5 9]
 [4 7 1 9 6 8 2 3 5]
 [8 6 9 5 2 3 4 7 1]
 [2 5 3 4 1 7 9 6 8]]
